In [2]:
import numpy as np 
import pandas as pd
import statsmodels.api as smf # python linear regression package! 
import matplotlib.pyplot as plt

### Calculating the OLS Estimator

In this notebook, we will calculate the **ordinary least squares (OLS) estimator**, along with other important statistical measures using our very own model built from scratch in Python! 

Ordinary least squares is a classic and often-used linear least-squares method for estimating parameters in a linear regression model. For example, let's suppose there are $n$ observations, $(x_{i}, y_{i})$, where $x_{i}$ is a vector of $r$ regressors such that $x_{i} = [x_{i1}, x_{i2}, ..., x_{ir}]$. 

Then, we can write $y_{i}$ as a function of the regressors in order to obtain a linear model: 

$$ y = x^{T}\beta + \varepsilon$$ 

where $\beta$ is a $r\times1$ vector of parameters that we estimate via the ordinary least squares, $y$ and $\varepsilon$ are $n\times1$ vectors of the outcome variables (predicted) and the errors. $x$ is an $n\times p$ matrix of the regressors. The estimate of the regression parameters, which is denoted as $\hat{\beta}$, is called the ordinary least squares estimator, and is given by the following equation:  

$$\hat{\beta} = (x^{T}x)^{-1}x^{T}y$$

Now, let's put this concept into practice by finding the OLS estimate for a sample regression! We will be using the same dataset as in the Ommitted Variable Bias notebook, with logwage as our outcome variable of interest, and state, age, education, and gender as regressors:

Thus, our regression model is: 

$$ log(wage_{i}) = \beta_{1}age_{i} + \beta_{2}educ_{i} + \beta_{3}female_{i} + \varepsilon_{i}$$

We can estimate the parameters $\beta$ with the statsmodels API as below: 

In [3]:
# read in data
ovb_data = pd.read_csv('ovb.csv')
ovb_data

,state,age,wagesal,imm,hispanic,black,asian,educ,wage,logwage,female,fedwkr,statewkr,localwkr
0,11,44,18000,0,0,0,0,14,9.109312,2.209297,1,1,0,0
1,11,39,18000,0,0,0,0,14,18.000000,2.890372,0,0,0,0
2,11,39,35600,0,0,0,0,12,17.115385,2.839978,0,0,0,1
3,11,39,8000,0,0,0,0,14,5.128205,1.634756,1,0,0,0
4,11,39,100000,0,0,0,0,16,38.461538,3.649659,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21902,95,36,125000,0,0,0,0,18,60.096154,4.095946,0,0,1,0
21903,95,38,70000,0,0,0,1,18,26.923077,3.292984,1,0,0,0
21904,95,43,48208,0,0,0,0,14,20.601709,3.025374,1,0,0,0
21905,95,43,75000,0,0,0,0,18,36.057692,3.585120,0,0,0,0


In [4]:
# define y and x sets
y = ovb_data['logwage'] 

# female is already a dummy variable that we can use. If there was also a male = 1 - female dummy variable, 
# then we would have to drop it from the regression due to perfect multicollinearity which would make the 
# inverse of the regressor matrix (design matrix) impossible to find. 
X = ovb_data[['age', 'educ', 'female']] 

In [5]:
# calculating the estimated coefficients, using the OLS estimator equation above
# NOTE: must convert Pandas Series to numpy arrays first

def ols_estimator(X, y):
    return np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)

beta_hats = ols_estimator(np.array(X), np.array(y))

regression_results = pd.DataFrame(index = ['age', 'educ', 'female'], 
                                      columns = ['Beta estimates'], 
                                      data=beta_hats)
regression_results

,Beta estimates
age,0.036325
educ,0.119588
female,-0.281464


### Calculating Variance and Standard Errors of Each Coefficient

Suppose we are interested in studying not only what the estimated coefficients on our regressors are, but also how "good" of a fit these estimators provide. One way that we can do this is my looking at the variance of our OLS estimator and the standard errors.

Assuming homoscedasticity, the variance of the OLS estimator is given by: 

$$Var(\hat{\beta}) = \hat{\sigma}^{2}_{\varepsilon}(x^{T}x)^{-1}$$

where $\hat{\sigma}^{2}_{\varepsilon}$ is the variance of the error term. How do we find the variance of this error term? Well, in linear regression, the unbiased estimator of variance of the error term is given by

$$ \hat{\sigma}^{2}_{\varepsilon} = \frac{1}{n-p-1}\sum_{i=1}^{n}{(y_{i} - \hat{y}_{i})^{2}} = \frac{1}{n-p-1}\sum_{i=1}^{n}{\hat{\varepsilon}_{i}} $$

where $n$ is the number of observations in the sample and $p$ is the number of regressors (parameters). It should also be noted that the summation term in the equation above is known as the **sum of squared residuals**.

Thus, $Var(\hat{\beta})$ then becomes a square matrix of shape $r \times r$, also known as the variance-covariance matrix. One property of this matrix is that it provides the covariance between pairs of elements in a random vector. Therefore, the matrix provided by $Var(\hat{\beta})$ contains the covariance between elements of $\hat{\beta}$. Furthermore, in order to find the **variance** of each individual coefficient, we must take each values on the diagonal of this matrix, since $Cov(\hat{\beta}_{i}, \hat{\beta}_{i}) = Var(\hat{\beta}_{i})$. To find the **standard error** of these values, we simply take the square root of the variance.

Let's try to calculate these values below using our data and OLS coefficients.

In [6]:
def error_term_variance(n, p, X, beta_hats, y): 
    # calculate the predicted value of y
    pred_y = np.dot(X, beta_hats) 
    # calculate the sum of squared residuals
    SSR = np.sum((y - pred_y)**2)
    # calculate the variance of the error term 
    return SSR/(n-p-1)

def variance_covariance_matrix(sigma_hat, X): 
    return sigma_hat * np.linalg.inv(np.dot(X.T, X))

n = X.shape[0]
p = X.shape[1]

sigma_hat = error_term_variance(n, p, np.array(X), beta_hats, np.array(y))
var_beta_hat = variance_covariance_matrix(sigma_hat, np.array(X))

print(var_beta_hat)

[[ 2.34945628e-07 -6.19448724e-07 -5.29988351e-07]
 [-6.19448724e-07  1.78907402e-06 -8.11555059e-07]
 [-5.29988351e-07 -8.11555059e-07  6.70192918e-05]]


Above is the displayed variance-covariance matrix -- taking the square root of values along the diagonal will provide us with the standard error for each term: 

In [7]:
# define function for these parts -- np.diagonal() is useful here! 
def coeff_var_from_var_matrix(matrix): 
    return matrix.diagonal()

def coeff_se_from_var_matrix(matrix): 
    return np.sqrt(matrix.diagonal())

variances = coeff_var_from_var_matrix(var_beta_hat)
standard_errors = coeff_se_from_var_matrix(var_beta_hat)

# add to the regression results!
regression_results['Variances'] = variances
regression_results['Standard Errors'] = standard_errors
regression_results

,Beta estimates,Variances,Standard Errors
age,0.036325,2.349456e-07,0.000485
educ,0.119588,1.789074e-06,0.001338
female,-0.281464,6.701929e-05,0.008187


### Finding the t-statistic and Confidence Interval for each Beta

Now that we have the OLS estimates for the coefficients of interest, we may be additionally interested in whether or not these estimates are statistically significant or not. Two popular ways of measuring the significance of our estimates are through a t-test, or through the use of confidence intervals. 

**T-test:**

For the purposes of a t-test in our regression, we want to find out whether or not our estimates support or reject the null hypothesis that the actual value is 0. More formally, we define two hypotheses about our estimated coeffiecients, a null hypothesis and an alternative hypothesis: 

$$ H_{0}: \hat{\beta}_{i} = 0 $$ 
$$ H_{1}: \hat{\beta}_{i} \neq 0 $$ 

The null hypothesis essentially asserts that our estimated coefficient is not statistically significant from 0, so we cannot say with certainty that any affect is attributed to the specific regressor that coefficient is estimated for. The alternative hypothesis asserts that the estimated coeffcient is, in fact, statistically significant from 0, so we can report, with some level of confidence, the effect of a regressor on the dependent variable. It must be noted that the value of 0 chosen is not a hard and fast rule -- in reality, we can perform a t-test comparing our estimates against another values $a$, of our choosing. Most statistical packages use 0 as the value in a t-test, so we proceed with it in this example. 

In order to calculate the t-statistic for each estimate, $\hat{\beta}_{i}$, we use the following formula and compare their absolute values (magnitude) to critical values from a normal distribution to determine at what confidence we can say that our estimates are significant: 

$$ t = \frac{\hat{\beta}_{i} - 0}{SE(\hat{\beta})} = \frac{\hat{\beta}_{i}}{SE(\hat{\beta})}$$

**Confidence Intervals:**

In a similar way to calculating the t-statistic, the confidence interval instead defines an interval based on our estimates and their standard errors for a set of values for which a hypothesis to a certain level of confidence cannot be rejected. For example, for a hypothesis test at the 10% level, the interval will have a 90% probability. of containing the true value of $\beta_{i}$.

If this interval contains 0, then we cannot say that our estimates are statistically significant from 0. If the interval does not contain 0, we can say that our estimates are statistically signficant from 0 at that level of confidence. 

Below is the formula for calculating the confidence interval for each estimate, $\hat{\beta}_{i}$ at the 95% confidence interval: 

$$ CI = [\hat{\beta}_{i} - 1.96 \times SE(\hat{\beta}_{i}), \hat{\beta}_{i} + 1.96 \times SE(\hat{\beta}_{i})]$$

The value of 1.96 corresponds to the z-score of a normal distribution for the 95% level of confidence. That is, 95% of the values in a normal distribution are contained between z-score values of +/- 1.96. This value will change for different confidence levels.

Now, let's put these concepts to practice and calculate the t-statistics and confidence intervals for our estimates at the 95% level:

In [8]:
# rounding the calculated values to avoid unnecessary messiness :)
def t_stat(beta_hat, se): 
    return np.round(beta_hat/se, 4)

def confidence_interval(beta_hat, se, z_score):
    cis = [] 
    for i in range(len(beta_hat)): 
        left_side = np.round(beta_hat[i] - z_score*se[i], 4)
        right_side = np.round(beta_hat[i] + 1.96*se[i], 4)
        cis.append((left_side, right_side)) 
    return cis

# using the beta hats and std. errors defined previously...
t_statistics = t_stat(beta_hats, standard_errors)
ci_90s = confidence_interval(beta_hats, standard_errors, 1.645) # 90% confidence
ci_95s = confidence_interval(beta_hats, standard_errors, 1.96)  # 95% confidence
ci_99s = confidence_interval(beta_hats, standard_errors, 2.576) # 99% confidence 

# add to the regression results!
regression_results['T-statistics'] = t_statistics
regression_results['90% Confidence Intervals'] = ci_90s
regression_results['95% Confidence Intervals'] = ci_95s
regression_results['99% Confidence Intervals'] = ci_99s
regression_results

,Beta estimates,Variances,Standard Errors,T-statistics,90% Confidence Intervals,95% Confidence Intervals,99% Confidence Intervals
age,0.036325,2.349456e-07,0.000485,74.9419,"(0.0355, 0.0373)","(0.0354, 0.0373)","(0.0351, 0.0373)"
educ,0.119588,1.789074e-06,0.001338,89.4073,"(0.1174, 0.1222)","(0.117, 0.1222)","(0.1161, 0.1222)"
female,-0.281464,6.701929e-05,0.008187,-34.3813,"(-0.2949, -0.2654)","(-0.2975, -0.2654)","(-0.3026, -0.2654)"


### Calculating $R^{2}$ and Adjusted $R^{2}$

**$R^{2}$:**

Another important statistic that is helpful to track when using OLS (and other types of regressions) is the coefficient of determination, or $R^{2}$. Essentially, the statistic calculates the proportion of the variation in the dependent variable that is explained by the independent variable(s). For example, if the $R^{2} = 0.49$ then we would say that 49% of the variation in the dependent variable in the data we are observing is explained by the independent regressors that we have included.

To calculate $R^{2}$, we must first find the values for the **sum of squared residuals (SSR)** and the **total sum of squares (TSS)**: 

$$ SSR = \sum_{i=1}^{n}{(y_{i} - \hat{y}_{i})^{2}}$$

$$ TSS \text{ (uncentered)} = \sum_{i=1}^{n}{(y_{i})^{2}} $$

$$ TSS \text{ (centered)} = \sum_{i=1}^{n}{(y_{i} - \bar{y})^{2}} $$

One way to think about the SSR is as the amount of unexplained variation in the dependent variable. The TSS can be viewed as total variation in the dependent variable (you can see that is formulation is very similar to the variance formula) and is thus proportional to the variance of the data. The reason for there being two separate types of TSS (centered vs. uncentered), is due to model specification. If we include a constant in our model specification, then we would use the centered TSS. If not, we would use the uncentered TSS since $E[y] = 0$ and thus $E[(y - E[y])^{2}] = E[y^{2}]$. 

To calculate $R^{2}$, we find, 
$$R^{2} = 1 - \frac{SSR}{TSS}$$

The lower bound is 0 for $R^{2}$, when SSR = TSS -- this means that the model is no better than a constant. The upper bound is 1 when SSR = 0, which means that the model perfectly explains all the variation in the data.

**Adjusted $R^{2}$:**

You might have noticed that the classic $R^{2}$ statistic suffers from one problem when we measure the "goodness" of fit for our model: it can only get better as we add more and more regressors as the statistic does not penalize adding more regressors even if they do not add much! This can be problematic, since we know that it often not beneficial to add as many regressors as we can (multicollinearity comes to mind). 

One way to correct for this is to calculate Adjusted $R^{2}$, which is a "stricter" statistic as it penalizes the model for extra regressors by replacing the SSR and TSS terms with degrees of freedom corrected terms: 

$$ \text{Adjusted } R^{2} = 1 - \frac{\frac{1}{n-p}SSR}{\frac{1}{n-1}TSS}$$

The idea behind these "df corrected" terms is that if we had $p$ regressors and a sample size of $n = p$, then you could perfectly predict each of the dependent variables. Thus, as we add more regressors, the numerator grows larger and the Adjusted $R^{2}$ will fall as a result.

Let's put this into practice and calculate these statistics for our OLS model!

**Note:** both $R^{2}$ and Adjusted $R^{2}$ are calculate for the whole model, not a single parameter estimate. 

In [30]:
def sum_of_squared_residuals(y, y_pred): 
    return np.sum((y - y_pred)**2)

def total_sum_of_squares_uncentered(y): 
    return np.sum((y)**2)

def total_sum_of_squares_centered(y): 
    return np.sum((y-np.mean(y))**2)

def r_squared(ssr, tss): 
    return 1 - ssr/tss

def adj_r_squared(ssr, tss, n, p):
    return 1 - (ssr/(n-p))/(tss/(n-1))

# calculate preliminary statistics

n = np.shape(X)[0]             # number of obs 
p = np.shape(X)[1]             # number of parameters
y_pred = np.dot(X, beta_hats)  # predicted y's 

ssr = sum_of_squared_residuals(y, y_pred)
tss = total_sum_of_squares_uncentered(y) 

# calculate r_squared 
r_2 = r_squared(ssr, tss) 
adj_r_2 = adj_r_squared(ssr, tss, n, p)

# add these to our regression results! 
regression_results['R-squared (uncentered)'] = np.ones(3)*r_2
regression_results['Adjusted R-squared (uncentered)'] = np.ones(3)*adj_r_2

regression_results

,Beta estimates,Variances,Standard Errors,T-statistics,90% Confidence Intervals,95% Confidence Intervals,99% Confidence Intervals,R-squared,Adjusted R-squared,R-squared (uncentered),Adjusted R-squared (uncentered)
age,0.036325,2.349456e-07,0.000485,74.9419,"(0.0355, 0.0373)","(0.0354, 0.0373)","(0.0351, 0.0373)",0.960947,0.960943,0.960947,0.960943
educ,0.119588,1.789074e-06,0.001338,89.4073,"(0.1174, 0.1222)","(0.117, 0.1222)","(0.1161, 0.1222)",0.960947,0.960943,0.960947,0.960943
female,-0.281464,6.701929e-05,0.008187,-34.3813,"(-0.2949, -0.2654)","(-0.2975, -0.2654)","(-0.3026, -0.2654)",0.960947,0.960943,0.960947,0.960943


There you have it! We have calculated a large number of helpful statistics for our very own OLS regression from scratch! Now that we've completed all the fun parts, lets see how we can very easily compute all of these statistics (and more) using the Statsmodels API. 

### The Easier Way: Statsmodels API

Now that we have found the estimates for each coefficient, we can confirm the accuracy of our calculations using the statsmodels API. 

Using the statsmodels API, we can easily find the OLS regression estimates with a couple lines of code -- along with some extra statistics that are useful when conducting a regression analysis. 

In [10]:
model1 = smf.OLS(y, X) 
res1 = model1.fit()
print(res1.summary())

                                 OLS Regression Results                                
Dep. Variable:                logwage   R-squared (uncentered):                   0.961
Model:                            OLS   Adj. R-squared (uncentered):              0.961
Method:                 Least Squares   F-statistic:                          1.797e+05
Date:                Fri, 12 Nov 2021   Prob (F-statistic):                        0.00
Time:                        18:01:50   Log-Likelihood:                         -20071.
No. Observations:               21907   AIC:                                  4.015e+04
Df Residuals:                   21904   BIC:                                  4.017e+04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Nice! We can see that the estimated regression coefficients are shown in the table above, along with some relevant statistics such as $R^{2}$, standard error, the t-statistic, and the confidence interval for each coefficient. Furthermore, comparing these results to our calculated regression coefficients above, we can see that the numbers are practically the same. 

The hope so far is that you are now comfortable with some of the math and intuition behind the common regression packages that can be used in Python. Let's package all of this together by now moving forward with actually **interpreting** the results of our regression.

### (In Progress) Interpreting the regression results

### Extra 1: How to add a constant term to the Statsmodels API

Below is an example of adding a constant when running a statsmodels OLS regression. We can see that the values for the $R^{2}$ and Adjusted $R^{2}$ are now equal to their "centered" versions.

In [38]:
X2 = smf.add_constant(X)
model2 = smf.OLS(y, X2) 
res2 = model2.fit()
print(res2.summary())

                            OLS Regression Results                            
Dep. Variable:                logwage   R-squared:                       0.248
Model:                            OLS   Adj. R-squared:                  0.248
Method:                 Least Squares   F-statistic:                     2410.
Date:                Fri, 12 Nov 2021   Prob (F-statistic):               0.00
Time:                        18:36:10   Log-Likelihood:                -19882.
No. Observations:               21907   AIC:                         3.977e+04
Df Residuals:                   21903   BIC:                         3.980e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1694      0.060     19.530      0.0

### Extra 2: OLS with just a constant

When we run OLS regressions with just a constant as the independent regressor, our estimated coefficient will end up just being the mean of the the observations of the dependent variable that we have. Below we demonstrate this: 

In [32]:
X3 = np.ones(len(X))
model3 = smf.OLS(y, X3) 
res3 = model3.fit()
print(res3.summary())

                            OLS Regression Results                            
Dep. Variable:                logwage   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Fri, 12 Nov 2021   Prob (F-statistic):                nan
Time:                        18:32:25   Log-Likelihood:                -23006.
No. Observations:               21907   AIC:                         4.601e+04
Df Residuals:                   21906   BIC:                         4.602e+04
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9816      0.005    638.095      0.0

In [37]:
print("The estimated OLS coefficient with just a constant is ", res3.params[0], 
      ". The mean of y is: ", np.mean(y), ". Thus, we can see that the two are the same!")

The estimated OLS coefficient with just a constant is  2.9816119732445805 . The mean of y is:  2.981611973244579 . Thus, we can see that the two are the same!
